# Step 3: Perform fit

As explained in the [previous step](2_generate_data), an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object behaves just like a mathematical function that takes a set of data points as an argument and returns a list of intensities (real numbers). At this stage, we want to optimize the parameters of the intensity model, so that it matches the distribution of our data sample. This is what we call 'performing a fit'.

```{note}
We first load the relevant data from the previous steps.
```

In [ ]:
import numpy as np
from expertsystem import io

from tensorwaves.physics.helicity_formalism.amplitude import IntensityBuilder
from tensorwaves.physics.helicity_formalism.kinematics import HelicityKinematics

In [ ]:
model = io.load_amplitude_model("amplitude_model_helicity.yml")
kinematics = HelicityKinematics.from_model(model)
data_sample = np.load("data_sample.npy")
phsp_sample = np.load("phsp_sample.npy")
intensity_builder = IntensityBuilder(model.particles, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity(model)
phsp_set = kinematics.convert(phsp_sample)
data_set = kinematics.convert(data_sample)

## 3.1 Define estimator

To perform a fit, you need to define an **estimator**. An estimator is a measure for the discrepancy between the intensity model and the data distribution to which you fit. In PWA, we usually use an *unbinned negative log likelihood estimator*, which can be created as follows with the [tensorwaves.estimator](tensorwaves.estimator) module:

In [ ]:
from tensorwaves.estimator import UnbinnedNLL

estimator = UnbinnedNLL(intensity, data_set)

The parameters that the [UnbinnedNLL](tensorwaves.estimator.UnbinnedNLL) carries, have been taken from the [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object and you'll recognize them from the YAML recipe file that we created in [Step 1](1_create_model):

In [ ]:
estimator.parameters

## 3.3 Optimize the intensity model

Now it's time to perform the fit! This is the part where the TensorFlow backend comes into play. For this reason, we suppress a few warnings:

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")

Starting the fit itself is quite simple: just create an optimizer instance of your choice, here [Minuit2](https://root.cern.ch/doc/master/Minuit2Page.html), and call its [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method to start the fitting process. Notice that the [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method requires a second argument. This is a mapping of parameter names that you want to fit to their initial values.

Let's first select a few of the parameters that we saw in [Step 3.1](#3.1-Define-estimator) and feed them to the optimizer to run the fit. Notice that we modify the parameters slightly to make the fit more interesting (we are running using a data sample that was generated with this very amplitude model after all).

In [ ]:
initial_parameters = {
    "Phase_J/psi(1S)_to_f(0)(1500)_0+gamma_1;f(0)(1500)_to_pi0_0+pi0_0;": 0.0,
    "Width_f(0)(500)": 0.1,
    "Mass_f(0)(980)": 0.9,
    "Mass_f(0)(1500)": 1.55,
    "Mass_f(0)(1710)": 1.8,
    "Width_f(0)(1710)": 0.3,
}

Recall that an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object is really just a [Function](tensorwaves.interfaces.Function) that computes the intensity for a certain dataset. This can be seen now nicely when we use these intensities as weights on the phase space sample and plot it together with the original dataset. Here, we look at the invariant mass distribution projection of the final states `3` and `4`, which, {ref}`as we saw before <usage/2_generate_data:2.4 Visualize kinematic variables>`, are the final state particles $\pi^0,\pi^0$.

Don't forget to use [update_parameters](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF.update_parameters) first!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def compare_model(
    variable_name, data_set, phsp_set, intensity_model, transform=lambda p: p, bins=100
):
    data = transform(data_set[variable_name])
    phsp = transform(phsp_set[variable_name])
    intensities = intensity_model(phsp_set)
    plt.hist(data, bins=bins, alpha=0.5, label="data", density=True)
    plt.hist(
        phsp,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="initial fit model",
        density=True,
    )
    plt.legend()

In [ ]:
intensity.update_parameters(initial_parameters)
compare_model("mSq_3_4", data_set, phsp_set, intensity, np.sqrt)

Finally, we create an [Optimizer](tensorwaves.interfaces.Optimizer) to [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) the model (which is embedded in the [Estimator](tensorwaves.interfaces.Estimator). Only the parameters that are given to the [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method are optimized.

In [ ]:
from tensorwaves.optimizer.minuit import Minuit2

minuit2 = Minuit2()
result = minuit2.optimize(estimator, initial_parameters)
result

```{admonition} Computation time ― around a minute in this case
---
class: dropdown
---
The computation time depends on the complexity of the model, number of data events, the size of the phase space sample, and the number of free parameters. This model is rather small and has but a few free parameters, so the optimization shouldn't take more than a few minutes.
```

Using the same method as above, we renew the parameters of the [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) and plot it again over the data distribution.

In [ ]:
intensity.update_parameters(initial_parameters)
compare_model("mSq_3_4", data_set, phsp_set, intensity, np.sqrt)

## 3.4 Export fit result

```{admonition} Not implemented yet
---
class: dropdown
---
This functionality has not yet been implemented. See [issue 99](https://github.com/ComPWA/tensorwaves/issues/99).
```

Optimizing an intensity model can take a long time, so it is important that you store the fit result in the end. We can simply [dump](json.dump) the file to [json](json):

In [ ]:
import json

with open("tensorwaves_fit_result.json", "w") as stream:
    json.dump(result, stream, indent=2)

...and [load](json.load) it back:

In [ ]:
with open("tensorwaves_fit_result.json") as stream:
    result = json.load(stream)
result